In [110]:
import tkinter as tk
from tkinter import messagebox, scrolledtext
from tkinter import ttk
from openpyxl import load_workbook, Workbook
import datetime

In [111]:
# Variable global para almacenar resultados
resultados_por_rol = {}
aplicativos = []
perfiles = []
nombre_archivo="archivo_generado.xlsx"

root = tk.Tk()
root.title("Busqueda en catalogo para codigos cargos")
root.geometry("1000x500")
root.resizable(True, True)

# Crear el Combobox para aplicativos
tk.Label(root, text="Aplicativo:").pack()
combo_aplicativos = ttk.Combobox(root)
combo_aplicativos.pack(fill="x", padx=10, pady=5)

# Campo para seleccionar perfil (ahora será el combobox para "palabra a buscar")
tk.Label(root, text="Palabra a buscar:").pack()
entry_palabra = ttk.Combobox(root)
entry_palabra.pack(fill="x", padx=10, pady=5)

In [112]:

def guardar_en_excel():
    palabra_buscada = entry_palabra.get().strip()
    aplicativo_excl = combo_aplicativos.get()
    if not resultados_por_rol:
        messagebox.showerror("Error", "No hay resultados para generar el reporte.")
        return
    
    wb = Workbook()
    ws = wb.active
    ws.title = "Reporte"

    # Determinar el número máximo de códigos de cargo
    max_codigos = max(len(codigos) for codigos in resultados_por_rol.values())

    encabezados = [f"Rol que contiene {palabra_buscada} en {aplicativo_excl}"] + [f"Código Cargo {i+1}" for i in range(max_codigos)]
    ws.append(encabezados)

    for rol_value, codigos in resultados_por_rol.items():
        fila = [rol_value]  
        
        fila.extend(codigos)

        # Asegurarse de que cada fila tenga el número correcto de columnas
        while len(fila) < max_codigos + 1:  
            fila.append("")  
        ws.append(fila)

    fecha_actual = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    nombre_archivo = f"CodigoRol_{fecha_actual}.xlsx"
    wb.save(nombre_archivo)
    messagebox.showinfo("Éxito", f"El reporte ha sido guardado como '{nombre_archivo}'.")


In [113]:
def cargar_aplicativos_y_perfiles():
    try:
        workbook = load_workbook(filename=nombre_archivo)
        sheet = workbook.worksheets[0]  # Acceder a la primera hoja
        max_row = sheet.max_row

        global aplicativos, perfiles
        aplicativos = set()
        perfiles = set()

        for i in range(2, max_row + 1):
            # Cargar aplicativos y limpiar las palabras
            if sheet["E" + str(i)].value:
                # Separar por delimitadores y agregar a un set para evitar duplicados
                palabras_aplicativo = [palabra.strip() for palabra in 
                                       sheet["E" + str(i)].value.replace(';', ',').split(',')]
                for palabra in palabras_aplicativo:
                    if palabra:
                        aplicativos.add(palabra)

            # Cargar perfiles y limpiar las palabras
            if sheet["G" + str(i)].value:
                palabras_perfil = [palabra.strip() for palabra in 
                                   sheet["G" + str(i)].value.replace(';', ',').split(',')]
                for palabra in palabras_perfil:
                    if palabra:
                        perfiles.add(palabra)

        # Actualizar los combobox en la GUI
        combo_aplicativos['values'] = list(aplicativos)
        entry_palabra['values'] = list(perfiles)

    except Exception as e:
        messagebox.showerror("Error", f"Ocurrió un error al cargar datos: {e}")

In [114]:
cargar_aplicativos_y_perfiles()
# Área de texto con scrollbar
frame_resultado = tk.Frame(root)
frame_resultado.pack(fill="both", expand=True, padx=10, pady=5)

resultado_text = scrolledtext.ScrolledText(frame_resultado, wrap="word", height=10)
resultado_text.pack(fill="both", expand=True)
resultado_text.config(state=tk.DISABLED)

# Botón para generar el reporte (inicialmente desactivado)
btn_generar_reporte = tk.Button(root, text="Generar Reporte", command=guardar_en_excel, state=tk.DISABLED)
btn_generar_reporte.pack(pady=5)

In [115]:
def buscar_datos():
    global resultados_por_rol  
    palabra_buscada = entry_palabra.get().strip()
    aplicativo_excl = combo_aplicativos.get()

    if not palabra_buscada or not aplicativo_excl:
        messagebox.showerror("Error", "Debe ingresar todos los parámetros")
        return

    try:
        # Cargar el archivo y obtener la primera hoja
        workbook = load_workbook(filename=nombre_archivo)
        sheet = workbook.worksheets[0]  # Acceder a la primera hoja

        rol = "B"  # Columna de rol
        aplicacion = "E"  # Columna de aplicativos
        perfil = "G"  # Columna de perfil
        codigocargo = "J"  # Columna de código de cargo

        max_row = sheet.max_row

        # Función para separar aplicativos y normalizar
        def separar_aplicativos(aplicativos_str):
            if not aplicativos_str:
                return []
            # Usar expresiones regulares para dividir por comas, punto y coma o espacios
            import re
            return [app.strip() for app in re.split(r'[;, ]+', aplicativos_str) if app.strip()]

        # Filtrar filas donde el aplicativo contenga la palabra buscada
        filas_con_aplicativo_y_rol = [
            i for i in range(2, max_row + 1)
            if (sheet[aplicacion + str(i)].value and 
                aplicativo_excl in separar_aplicativos(sheet[aplicacion + str(i)].value) and 
                sheet[rol + str(i)].value is not None)
        ]

        # Filtrar filas que contengan la palabra buscada en "perfil"
        filas_con_palabra = [
            i for i in filas_con_aplicativo_y_rol
            if sheet[perfil + str(i)].value and 
               palabra_buscada in sheet[perfil + str(i)].value.strip()
        ]

        # Agrupar resultados por rol
        resultados_por_rol.clear()  # Limpiar resultados previos

        for i in filas_con_palabra:
            rol_value = sheet[rol + str(i)].value
            cod_cargo = sheet[codigocargo + str(i)].value
            if rol_value not in resultados_por_rol:
                resultados_por_rol[rol_value] = []
            resultados_por_rol[rol_value].append(cod_cargo)

        resultado_text.config(state=tk.NORMAL)
        resultado_text.delete("1.0", tk.END)

        if resultados_por_rol:
            resultado_text.insert(tk.END, f"Aplicativo: {aplicativo_excl}\nPerfil: {palabra_buscada.capitalize()}\n\n")
            for rol_value, codigos in resultados_por_rol.items():
                resultado_text.insert(tk.END, f"--------------\nRol: {rol_value}\n\n" + "\n".join(f"Código Cargo: {codigo}" for codigo in codigos) + "\n\n")

            btn_generar_reporte.config(state=tk.NORMAL)

        else:
            resultado_text.insert(tk.END, "No se encontraron resultados")
            btn_generar_reporte.config(state=tk.DISABLED)  # Desactivar botón

        resultado_text.config(state=tk.DISABLED)

    except Exception as e:
        messagebox.showerror("Error", f"Ocurrió un error: {e}")

In [116]:
tk.Button(root, text="Buscar", command=buscar_datos).pack(pady=5)

In [117]:
root.mainloop()